In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import kraft

In [3]:
di = "../raw/"

## Check raw

In [4]:
na_pa = {}

for pa in kraft.path.get_child(di):

    if pa[-3:] != ".md":

        continue

    na = pa.split("/")[-1][: -len(".md")]

    assert na == kraft.string.untitle(na), na

    na_pa[na] = pa

    se_ = [".", "<", ">"]

    with open(pa) as io:

        for ie, li in enumerate(io):

            li = li.strip()

            er = "{}: {}".format(na, li)

            if ie % 2 == 1:

                assert li == "", er

            elif li[0] == "#":

                se = li[2]

                assert se == se_.pop(0), er

            elif se == ".":

                assert li[-1] == ".", er

len(na_pa)

507

## Read raw

In [5]:
na_se_li_ = {}

for na, pa in na_pa.items():

    se_li_ = {
        ".": [],
        "<": [],
        ">": [],
    }

    with open(pa) as io:

        for li in io.readlines()[0::2]:

            li = li.strip()

            if li[0] == "#":

                se = li[2]

            else:

                se_li_[se].append(li)

    na_se_li_[na] = se_li_

## Link .md

In [6]:
for se_li_ in na_se_li_.values():

    se_li_["md<"] = []

    se_li_[">md"] = []

for na, se_li_ in na_se_li_.items():
    
    for li in list(se_li_["<"]):

        li2 = kraft.string.untitle(li)

        if li2 in na_se_li_:
            
            print(na, li2)

            se_li_["<"].remove(li)

            na_se_li_[li2][">md"].append(na)

    for li in list(se_li_[">"]):

        li2 = kraft.string.untitle(li)

        if li2 in na_se_li_:

            se_li_[">"].remove(li)

            se_li_[">md"].append(li2)

            na_se_li_[li2]["md<"].append(na)

## Check collision

In [7]:
def combine_future(se_li_):

    return se_li_[">"] + kraft.iterable.flatten(
        [na_se_li_[li][">"] for li in se_li_[">md"]]
    )

In [8]:
import pandas as pd

In [9]:
for na, se_li_ in na_se_li_.items():

    for li, n_re in pd.value_counts(combine_future(se_li_)).items():

        if 1 < n_re:

            so_ = [na]

            so_ += [li2 for li2 in se_li_[">md"] if li in na_se_li_[li2][">"]]

            # print("{}:\n\t{}\n".format(li, "\n\t".join(so_)))

## Read feature

In [10]:
fe_pr = {}

for pa in kraft.path.list("../feature/"):
    
    pr = []
    
    for na in pa.split("/"):
        
        fi = na.split("_")[0]
        
        if fi.isdigit():
            
            pr.append(int(fi))
        
    for ie, fe in enumerate(pd.read_csv(pa, "\t", index_col=0, comment="#").index):
        
        fe_pr[fe] = pr + [ie]

## Check extra feature 

In [11]:
fe_ = []

for se_li_ in na_se_li_.values():

    fe_ += se_li_["<"] + se_li_[">"]

In [12]:
ex_ = []

for fe in fe_pr.keys():

    if fe not in fe_:

        ex_.append(fe)

print(len(ex_))

print("\n".join(ex_))

30
Bradycardia
Tachycardia
Bradypnea
Hyperventilation
Hypoxemia
Hypopigmentation
Nail bed hemorrhage
Clubbing
Holo-systolic murmur
Diastolic murmur
Aortic regurgitation
Native-American
African-American
Epistasis
Recurrent chest pain
Chest pain radiating to arm or jaw
Cough
Burning abdominal pain
Left upper quadrant abdominal pain
Bloody diarrhea
Oliguria
Urinary urgency
Dysmenorrhea
Pregnancy
Irregularly irregular
Long QT
Inverted T
Antidepressant
B blocker
Tropic


## Check missing feature

In [13]:
mi_ = []

for na, se_li_ in na_se_li_.items():

    for li in se_li_["<"] + se_li_[">"]:

        if li not in fe_pr:

            mi_.append(li)

mi_ = set(mi_)

print(len(mi_))

print("\n".join(mi_))

243
Loss of pressure
Cerebral hemorrhage
Hypercalciuria
Pica
Irregularly irregular rhythm
osteitis fibrosa cysica
Multiple endocrine neoplasia 2A
Arm paresthesia
Hyperresonance
Asymmetric muscle weakness
Addision disease
Loss of pain
Worse at night
Long PR
Hypocalciuria
tricuspid regurgitation
Hyperphosphaturia
High hemoglobin A1C
Cerebrovascular disease
Cardiac hypertrophy
Fine crackle
Pressure chest pain
Ventricular aneurysm
Cerebral aneurysm
Right ventricle hypertrophy
Atrioventricular septal defect
Unilateral hyperresonance
hyperphosphatemia
Dullness to percussion
Chest pain radiating to either arm or jaw
Respiratory tract infction
Ion channelopathy
Electric alteran
Psychiatric disease
Overriding aorta
Infectino
Systolic regurgitation murmur
Septic arterial embolism
Hypomagnesimia
Non-pleuritic chest pain
Atria beat faster than ventricles
Macroalbuminuria
Endemic in tropics
Wet beriberi
Ondansetron
Osteolytic metastasis
Pulmonary edema
Duodenal ulcer
Assymmetric arm blood pressure


## Rewrite

In [14]:
def prioritize(fe):

    return fe_pr.get(fe, [0]) + [fe]

In [15]:
def sort(li_):

    return sorted(li_, key=prioritize)

In [16]:
for na, se_li_ in na_se_li_.items():

    with open(na_pa[na], "w") as io:

        ti_li_ = {
            ".": se_li_["."],
            "<": se_li_["<"],
            ">": se_li_[">"] + [kraft.string.title(li).upper() for li in se_li_[">md"]],
        }

        for ti, li_ in ti_li_.items():

            if ti != ".":

                io.write("\n")

            io.write("# {}\n".format(ti))

            if 0 < len(li_):

                io.write("\n")

                io.write("\n\n".join(sort(li_)) + "\n")

## Clean

In [17]:
import os
import shutil

In [18]:
di2 = "../clean/"

if os.path.isdir(di2):

    shutil.rmtree(di2)

In [19]:
for na, se_li_ in na_se_li_.items():

    pa = na_pa[na].replace(di, di2)

    kraft.path.make(pa, pr=False)

    with open(pa, "w") as io:

        ti_li_ = {
            "Comment": se_li_["."],
            "Epidemiology": [],
            "Cause": [kraft.string.title(li) for li in se_li_["md<"]],
            "Symptom": [],
            "Sign": [],
            "Diagnostic": [],
            # "Treatment and Management": [],
        }

        for li in se_li_["<"]:

            try:
                nu_ = number(fe_pr[li])
            except:
                continue

            if [2, 0, 0] <= nu_ < [6, 0, 0]:

                ti_li_["Epidemiology"].append(li)

        for li in combine_future(se_li_):

            try:
                nu_ = number(fe_pr[li])
            except:
                continue

            if [1, 0, 0] <= nu_ < [2, 0, 0]:

                ti_li_["Symptom"].append(li)

            elif [6, 0, 0] <= nu_ < [7, 0, 0]:

                ti_li_["Sign"].append(li)

            elif [7, 0, 0] <= nu_:

                ti_li_["Diagnostic"].append(li)

        for ti, li_ in ti_li_.items():

            if 0 < len(li_):

                if ti != "Comment":

                    io.write("\n")

                io.write("# {}\n".format(ti))

                io.write("\n")

                io.write("\n\n".join(sort(li_)) + "\n")